In [ ]:
# default_exp models.ResNetPlus

# ResNetPlus

> This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com based on:
* Wang, Z., Yan, W., & Oates, T. (2017, May). Time series classification from scratch with deep neural networks: A strong baseline. In 2017 international joint conference on neural networks (IJCNN) (pp. 1578-1585). IEEE.
* Fawaz, H. I., Forestier, G., Weber, J., Idoumghar, L., & Muller, P. A. (2019). Deep learning for time series classification: a review. Data Mining and Knowledge Discovery, 33(4), 917-963.
* Official ResNet TensorFlow implementation: https://github.com/hfawaz/dl-4-tsc
* 👀 kernel filter size 8 has been replaced by 7 (I believe it's a bug since even kernels are not commonly used in practice)

In [ ]:
#export
from fastai.layers import *
from tsai.imports import *
from tsai.models.layers import *
from tsai.models.utils import *
torch.set_num_threads(cpus)

In [ ]:
# export
class ResBlockPlus(Module):
    def __init__(self, ni, nf, ks=[7, 5, 3], coord=False, separable=False, bn_1st=True, zero_norm=False, sa=False, se=None, act=nn.ReLU, act_kwargs={}):
        self.convblock1 = ConvBlock(
            ni, nf, ks[0], coord=coord, separable=separable, bn_1st=bn_1st, act=act, act_kwargs=act_kwargs)
        self.convblock2 = ConvBlock(
            nf, nf, ks[1], coord=coord, separable=separable, bn_1st=bn_1st, act=act, act_kwargs=act_kwargs)
        self.convblock3 = ConvBlock(
            nf, nf, ks[2], coord=coord, separable=separable, zero_norm=zero_norm, act=None)
        self.se = SEModule1d(
            nf, reduction=se, act=act) if se and nf//se > 0 else noop
        self.sa = SimpleSelfAttention(nf, ks=1) if sa else noop
        self.shortcut = BN1d(ni) if ni == nf else ConvBlock(
            ni, nf, 1, coord=coord, act=None)
        self.add = Add()
        self.act = act(**act_kwargs)

        self._init_cnn(self)

    def _init_cnn(self, m):
        if getattr(self, 'bias', None) is not None:
            nn.init.constant_(self.bias, 0)
        if isinstance(self, (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear)):
            nn.init.kaiming_normal_(self.weight)
        for l in m.children():
            self._init_cnn(l)

    def forward(self, x):
        res = x
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.se(x)
        x = self.sa(x)
        x = self.add(x, self.shortcut(res))
        x = self.act(x)
        return x


@delegates(ResBlockPlus.__init__)
class ResNetPlus(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, nf=64, sa=False, se=None, fc_dropout=0., concat_pool=False,
                 flatten=False, custom_head=None, y_range=None, **kwargs):

        resblock1 = ResBlockPlus(c_in,   nf,     se=se,   **kwargs)
        resblock2 = ResBlockPlus(nf,     nf * 2, se=se,   **kwargs)
        resblock3 = ResBlockPlus(nf * 2, nf * 2, sa=sa, **kwargs)
        backbone = nn.Sequential(resblock1, resblock2, resblock3)
        
        self.head_nf = nf * 2
        if flatten:
            assert seq_len is not None, "you need to pass seq_len when flatten=True"
            self.head_nf *= seq_len
        if custom_head is not None:
            head = custom_head(self.head_nf, c_out)
        else:
            head = self.create_head(self.head_nf, c_out, flatten=flatten,
                                         concat_pool=concat_pool, fc_dropout=fc_dropout, y_range=y_range)
        super().__init__(OrderedDict([('backbone', backbone), ('head', head)]))
            
    def create_head(self, nf, c_out, flatten=False, concat_pool=False, fc_dropout=0., y_range=None, **kwargs):
        layers = [Flatten()] if flatten else []
        if concat_pool:
            nf = nf * 2
        layers = [GACP1d(1) if concat_pool else GAP1d(1)]
        if fc_dropout:
            layers += [nn.Dropout(fc_dropout)]
        layers += [nn.Linear(nf, c_out)]
        if y_range:
            layers += [SigmoidRange(*y_range)]
        return nn.Sequential(*layers)

In [ ]:
from tsai.models.layers import Swish
xb = torch.rand(2, 3, 4)
test_eq(ResNetPlus(3,2)(xb).shape, [xb.shape[0], 2])
test_eq(ResNetPlus(3,2,coord=True, separable=True, bn_1st=False, zero_norm=True, act=Swish, act_kwargs={}, fc_dropout=0.5)(xb).shape, [xb.shape[0], 2])
test_eq(count_parameters(ResNetPlus(3, 2)), 479490) # for (3,2)

In [ ]:
from tsai.models.ResNet import *
test_eq(count_parameters(ResNet(3, 2)), count_parameters(ResNetPlus(3, 2))) # for (3,2)

In [ ]:
m = ResNetPlus(3, 2, zero_norm=True, coord=True, separable=True)
print('n_params:', count_parameters(m))
print(m)
print(check_weight(m, is_bn)[0])

n_params: 114820
ResNetPlus(
  (backbone): Sequential(
    (0): ResBlockPlus(
      (convblock1): ConvBlock(
        (0): AddCoords1d()
        (1): SeparableConv1d(
          (depthwise_conv): Conv1d(4, 4, kernel_size=(7,), stride=(1,), padding=(3,), groups=4, bias=False)
          (pointwise_conv): Conv1d(4, 64, kernel_size=(1,), stride=(1,), bias=False)
        )
        (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU()
      )
      (convblock2): ConvBlock(
        (0): AddCoords1d()
        (1): SeparableConv1d(
          (depthwise_conv): Conv1d(65, 65, kernel_size=(5,), stride=(1,), padding=(2,), groups=65, bias=False)
          (pointwise_conv): Conv1d(65, 64, kernel_size=(1,), stride=(1,), bias=False)
        )
        (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU()
      )
      (convblock3): ConvBlock(
        (0): AddCoords1d()
        (1): SeparableConv1d(
      

In [ ]:
#hide
from tsai.imports import *
from tsai.export import *
nb_name = get_nb_name()
create_scripts(nb_name);

<IPython.core.display.Javascript object>

101b_models.ResNetPlus.ipynb saved at 2022-02-28 13:04:46.
Converted 101b_models.ResNetPlus.ipynb.


Correct conversion! 😃
Total time elapsed 0.121 s
Monday 28/02/22 13:04:50 CET
